# Fusionner les lexèmes de plusieurs kalabas

- par exemple pour Mahira et Agathos

In [1]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle
#from cellbell import ding

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [2]:
%store -r numerosKalaba
numeroKalaba="21-K%d"%numerosKalaba[0]
print numeroKalaba

21-K4


## Ouvrir les deux lexiques

In [3]:
home = expanduser("~")
serieTarget=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%"21-K1"
serieSource=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%"21-K2"

with open(serieTarget+"Stems.yaml", 'r') as stream:
    lexTarget=yaml.safe_load(stream)
with open(serieSource+"Stems.yaml", 'r') as stream:
    lexSource=yaml.safe_load(stream)

## Générer des radicaux nouveaux
- sur la base de la fréquence des consonnes
- qui ne soient pas dans refStems

In [4]:
from numpy.random import choice

consonnesFreq=[10,10,10,10,10,10,8,8,6,7,7,7,7,5,5,5,5,8,8,3,3]
consonnesTot=sum(consonnesFreq)
consonnesProb=[float(k)/consonnesTot for k in consonnesFreq]


with open(serieTarget+"Phonology.yaml", 'r') as stream:
    phonology=yaml.safe_load(stream)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [5]:
def randomCVCVC(refStems=[]):
    result=""
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    if result not in refStems:
        return result
    else:
        return randomCVCVC(refStems)

In [6]:
randomCVCVC()

'dumot'

## Récuperer les informations lexicales

In [7]:
def getLexemes(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element]) is list:
#             formesCle=", ".join()
            formes[element]=stemStructure[element]
        else:
            formes.update(getLexemes(stemStructure[element]))
    return formes

In [8]:
def getLemmes(stemStructure):
    lexemes=getLexemes(stemStructure)
    result=[v[0] for k,v in lexemes.items()]
    return result

In [9]:
def getFrench(stemStructure):
    lexemes=getLexemes(stemStructure)
    result=[v for k,v in lexemes.items()]
    return result

In [10]:
def getStems(stemStructure):
    lexemes=getLexemes(stemStructure)
    result=[k for k,v in lexemes.items()]
    return result    

In [11]:
stemsSource=getStems(lexSource)
stemsTarget=getStems(lexTarget)

## Ajout des lexèmes manquants

In [12]:
for cat in lexSource:
    if cat!="PRO":
        frSource=getFrench(lexSource[cat])
        missingLexemes=set(getLemmes(lexSource[cat]))-set(getLemmes(lexTarget[cat]))

        stemsTargetNew=stemsTarget[:]
        for l in frSource:
            if l[0] in missingLexemes:
                print l
                radical=randomCVCVC(stemsTargetNew)
                stemsTargetNew.append(radical)
                lexTarget[cat][radical]=l
        
# lexAgathos    

['disparu', 'disparus', 'disparue', 'disparues']
['dernier', 'derniers', u'derni\xe8re', u'derni\xe8res']
['nombreux', 'nombreuses']
['vilain', 'vilains', 'vilaine', 'vilaines']
['nouveau', 'nouveaux', 'nouvelle', 'nouvelles']
['sept']
['six']
['autre', 'autres']
['deux']
['vert', 'verts', 'verte', 'vertes']
[u'troisi\xe8me', u'troisi\xe8mes']
[u'h\xe9ro\xefque', u'h\xe9ro\xefques']
[u'm\xe9chant', u'm\xe9chants', u'm\xe9chante', u'm\xe9chantes']
['cinq']
['treize']
['porter', 'porte', 'portent', 'portait', 'portaient', 'porta', u'port\xe8rent']
[u'br\xfbler', u'br\xfble', u'br\xfblent', u'br\xfblait', u'br\xfblaient', u'br\xfbla', u'br\xfbl\xe8rent']
['faire', 'fait', 'font', 'faisait', 'faisaient', 'fit', 'firent']
['raconter', 'raconte', 'racontent', 'racontait', 'racontaient', 'raconta', u'racont\xe8rent']
['manquer', 'manque', 'manquent', 'manquait', 'manquaient', 'manqua', u'manqu\xe8rent']
['placer', 'placE', 'placent', u'pla\xe7ait', u'pla\xe7aient', u'pla\xe7a', u'plac\xe8rent

## Stats sur les phonèmes dans le lexique

In [13]:
def statsCV(lstStems):
    cons={}
    vocs={v:0 for v in "aeiou"}
    for s in lstStems:
        for c in s:
            if c not in vocs: 
                if c not in cons: cons[c]=0
                cons[c]+=1
            else:
                vocs[c]+=1
    return cons,vocs

In [14]:
stems=getStems(lexSource)
statsCV(stems)

({'D': 28,
  'N': 26,
  'S': 37,
  'T': 35,
  'Z': 30,
  'b': 40,
  'd': 44,
  'f': 39,
  'g': 59,
  'j': 17,
  'k': 50,
  'l': 47,
  'm': 45,
  'n': 43,
  'p': 57,
  'r': 52,
  's': 34,
  't': 61,
  'v': 28,
  'w': 11,
  'z': 13},
 {'a': 135, 'e': 105, 'i': 89, 'o': 133, 'u': 120})

## Sauvegarde du nouveau lexique

In [15]:
with open(serieTarget+'fusedStems.yaml', 'w') as output:
    yaml.dump(lexTarget, output, default_flow_style=False,allow_unicode=True)

In [16]:
with open(serieTarget+"fusedStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [17]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  DamuS: [autre, autres]
  DuTog: [méchant, méchants, méchante, méchantes]
  NaZad: [sept]
  Sogeges: [terrible, terribles]
  Sozup: [différent, différents, différente, différentes]
  Zuges: [disparu, disparus, disparue, disparues]
  baZowaN: [bas, basse, basses]
  banaN: [vert, verts, verte, vertes]
  foder: [jaune, jaunes]
  funib: [treize]
  gadin: [maigre, maigres]
  genil: [deux]
  guNomaS: [gros, grosse, grosses]
  jemibot: [profond, profonds, profonde, profondes]
  joZogiN: [courageux, courageuse, courageuses]
  kaZot: [nombreux, nombreuses]
  kavijot: [effrayé, effrayés, effrayée, effrayées]
  kizow: [noir, noire, noirs, noires]
  ledaf: [troisième, troisièmes]
  lopiS: [dernier, derniers, dernière, dernières]
  milapeS: [grand, grande, grands, grandes]
  nebus: [vilain, vilains, vilaine, vilaines]
  nemiT: [blanc, blanche, blancs, blanches]
  poleS: [furieux, furieuse, furieuses]
  rumiT: [rouge, rouges]
  ruzet: [petit, petite, petits, petites]
  sejin: [quatre]
  sunuf:

In [18]:
with open(serieTarget+'fusedStems.yaml', 'w') as output:
    output.write(yamlText)